## Resnet 详解

### 论文理解
- 论文地址 https://arxiv.org/pdf/1512.03385.pdf

- 实验中发现的不足
![title](./img/11.PNG)

- 思想的启发
![title](./img/12.PNG)

- 网络对比
![title](./img/13.PNG)

- 结果
![title](./img/14.PNG)
- 遗留问题 （虚线是训练集， 实线是预测集）
![title](./img/15.PNG)

### 模型结构梳理

![title](./img/17.PNG)
![title](./img/18.PNG)

### 基础参数配置

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # 这一行注释掉就是使用gpu，不注释就是使用cpu

In [2]:
# 最大迭代次数
EPOCHS = 10

# 每次批处理数量
BATCH_SIZE = 32

# 多少分类
NUM_CLASSES = 3

# 照片高度
image_height = 32

# 照片宽度
image_width = 32

# 颜色通道数量
channels = 3

# 保存网络模型地址
save_model_dir = "saved_model/model"

# 数据地址
dataset_dir = "dataset/"
train_dir = dataset_dir + "train"
valid_dir = dataset_dir + "valid"
test_dir = dataset_dir + "test"

### 加载一些模块

In [3]:
import tensorflow as tf
import pathlib
import math
import warnings
warnings.filterwarnings("ignore")

### GPU设置

In [4]:
gpus = tf.config.list_physical_devices("GPU")
print(gpus)
if gpus:
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    
    tf.config.set_visible_devices([gpu0],"GPU") 

[]


### 数据切分

In [5]:
import os
import random

# 数据随机化
import shutil


class SplitDataset():
    def __init__(self, dataset_dir, saved_dataset_dir, train_ratio=0.6, test_ratio=0.2, show_progress=False):
        self.dataset_dir = dataset_dir
        self.saved_dataset_dir = saved_dataset_dir
        self.saved_train_dir = saved_dataset_dir + "/train/"
        self.saved_valid_dir = saved_dataset_dir + "/valid/"
        self.saved_test_dir = saved_dataset_dir + "/test/"


        self.train_ratio = train_ratio
        self.test_radio = test_ratio
        self.valid_ratio = 1 - train_ratio - test_ratio

        self.train_file_path = []
        self.valid_file_path = []
        self.test_file_path = []

        # 索引和标签的映射表
        self.index_label_dict = {}

        # 是否展现过程
        self.show_progress = show_progress

        # 数据存放位置
        if not os.path.exists(self.saved_train_dir):
            os.mkdir(self.saved_train_dir)
        if not os.path.exists(self.saved_test_dir):
            os.mkdir(self.saved_test_dir)
        if not os.path.exists(self.saved_valid_dir):
            os.mkdir(self.saved_valid_dir)


    def _get_label_names(self):
        # 把所有目录作为标签名称
        label_names = []
        for item in os.listdir(self.dataset_dir):
            item_path = os.path.join(self.dataset_dir, item)
            if os.path.isdir(item_path):
                label_names.append(item)
        return label_names

    def _get_all_file_path(self):
        
        all_file_path = []
        
        #得到各个类别
        for index, file_type in enumerate(self._get_label_names()):
            
            #给每个类别一个索引
            self.index_label_dict[index] = file_type
            
            #当前类别路径
            type_file_path = os.path.join(self.dataset_dir, file_type)
            
            file_path = []
            
            #遍历当前类别里的所有样本
            for file in os.listdir(type_file_path):
                single_file_path = os.path.join(type_file_path, file)
                file_path.append(single_file_path)
            
            # 获取一个二维 [[]] 列表，第一维列表的索引代表 第二维度列表的文件路径的类别
            all_file_path.append(file_path)
            
        return all_file_path


    def _split_dataset(self):
        all_file_paths = self._get_all_file_path()
        
        #每个类别分别做划分
        for index in range(len(all_file_paths)):
            
            #当前类别所有样本路径
            file_path_list = all_file_paths[index]
            file_path_list_length = len(file_path_list)
            random.shuffle(file_path_list)
            
            #随机选择训练和测试样本
            train_num = int(file_path_list_length * self.train_ratio)
            test_num = int(file_path_list_length * self.test_radio)

            self.train_file_path.append([self.index_label_dict[index], file_path_list[: train_num]])
            
            self.test_file_path.append([self.index_label_dict[index], file_path_list[train_num:train_num + test_num]])
            
            self.valid_file_path.append([self.index_label_dict[index], file_path_list[train_num + test_num:]])

    def _copy_files(self, type_path, type_saved_dir):
        for item in type_path:
            src_path_list = item[1]
            dst_path = type_saved_dir + "%s/" % (item[0])
            if not os.path.exists(dst_path):
                os.mkdir(dst_path)
            for src_path in src_path_list:
                shutil.copy(src_path, dst_path)
                if self.show_progress:
                    print("Copying file "+src_path+" to "+dst_path)
                    
    def start_splitting(self):
        # 主函数入口
        self._split_dataset()
        self._copy_files(type_path=self.train_file_path, type_saved_dir=self.saved_train_dir)
        self._copy_files(type_path=self.valid_file_path, type_saved_dir=self.saved_valid_dir)
        self._copy_files(type_path=self.test_file_path, type_saved_dir=self.saved_test_dir)

In [6]:
split_dataset = SplitDataset(dataset_dir="original_dataset",
                             saved_dataset_dir="dataset",
                             show_progress=True)
split_dataset.start_splitting()

Copying file original_dataset/dogs/dogs_00169.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00822.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00510.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00905.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00665.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00852.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00309.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00231.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00981.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00668.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00238.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00516.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00131.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00752.jpg t

Copying file original_dataset/dogs/dogs_00755.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00725.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00285.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00858.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00262.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00134.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00374.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00212.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00360.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00624.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00202.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00017.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00457.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00145.jpg t

Copying file original_dataset/dogs/dogs_00140.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00695.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00813.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00801.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00018.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00284.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00831.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00612.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00879.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00149.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00278.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00843.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00049.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00474.jpg t

Copying file original_dataset/dogs/dogs_00357.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00241.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00271.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00117.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00254.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00518.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00865.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00165.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00463.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00560.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00684.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00557.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00693.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00597.jpg t

Copying file original_dataset/dogs/dogs_00703.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00329.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00458.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00106.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00799.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00430.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00052.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00579.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00154.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00190.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00083.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00081.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00902.jpg to dataset/train/dogs/
Copying file original_dataset/dogs/dogs_00854.jpg t

...

Copying file original_dataset/dogs/dogs_00656.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00654.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00026.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00340.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00862.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00720.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00472.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00444.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00525.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00092.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00886.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00143.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00895.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00065.jpg to dataset/tes

Copying file original_dataset/dogs/dogs_00569.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00707.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00133.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00287.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00660.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00246.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00107.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00818.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00535.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00418.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00004.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00709.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00139.jpg to dataset/test/dogs/
Copying file original_dataset/dogs/dogs_00606.jpg to dataset/tes

Copying file original_dataset/cats/cats_00112.jpg to dataset/test/cats/
Copying file original_dataset/cats/cats_00125.jpg to dataset/test/cats/
Copying file original_dataset/cats/cats_00656.jpg to dataset/test/cats/
Copying file original_dataset/cats/cats_00596.jpg to dataset/test/cats/
Copying file original_dataset/cats/cats_00627.jpg to dataset/test/cats/
Copying file original_dataset/cats/cats_00410.jpg to dataset/test/cats/
Copying file original_dataset/cats/cats_00851.jpg to dataset/test/cats/
Copying file original_dataset/cats/cats_00645.jpg to dataset/test/cats/
Copying file original_dataset/cats/cats_00936.jpg to dataset/test/cats/
Copying file original_dataset/cats/cats_00931.jpg to dataset/test/cats/
Copying file original_dataset/cats/cats_00265.jpg to dataset/test/cats/
Copying file original_dataset/cats/cats_00399.jpg to dataset/test/cats/
Copying file original_dataset/cats/cats_00233.jpg to dataset/test/cats/
Copying file original_dataset/cats/cats_00074.jpg to dataset/tes

Copying file original_dataset/panda/panda_00175.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00048.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00032.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00339.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00823.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00418.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00090.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00720.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00316.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00477.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00792.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00569.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00302.jpg to dataset/test/panda/
Copying file original_dat

Copying file original_dataset/panda/panda_00660.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00337.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00311.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00936.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00134.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00695.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00327.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00420.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00797.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00645.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00502.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00636.jpg to dataset/test/panda/
Copying file original_dataset/panda/panda_00416.jpg to dataset/test/panda/
Copying file original_dat

### 生成训练数据

In [7]:
def load_and_preprocess_image(img_path):
    # 读取照片
    img_raw = tf.io.read_file(img_path)
    # 解码照片
    img_tensor = tf.image.decode_jpeg(img_raw, channels=channels)
    # 照片resize
    img_tensor = tf.image.resize(img_tensor, [image_height, image_width])
    img_tensor = tf.cast(img_tensor, tf.float32)
    # 归一化
    img = img_tensor / 255.0
    return img

def get_images_and_labels(data_root_dir):
    # 得到所有图像路径
    data_root = pathlib.Path(data_root_dir)
    
    all_image_path = [str(path) for path in list(data_root.glob('*/*'))]
    # 得到标签名字
    label_names = sorted(item.name for item in data_root.glob('*/'))
    # 例如：{'cats': 0, 'dogs': 1, 'panda': 2}
    label_to_index = dict((index, label) for label, index in enumerate(label_names))
    # 每一个图像对应的标签
    all_image_label = [label_to_index[pathlib.Path(single_image_path).parent.name] for single_image_path in all_image_path]

    return all_image_path, all_image_label


def get_dataset(dataset_root_dir):
    
    # 获取每个照片对应的标签
    all_image_path, all_image_label = get_images_and_labels(data_root_dir=dataset_root_dir)
    
    # 通过自定义的load_and_preprocess_image函数，从照片路径中读取照片做为数据集
    image_dataset = tf.data.Dataset.from_tensor_slices(all_image_path).map(load_and_preprocess_image)
    
    # 获取标签数据集
    label_dataset = tf.data.Dataset.from_tensor_slices(all_image_label)
    
    # 让照片和标签一一对应
    dataset = tf.data.Dataset.zip((image_dataset, label_dataset))
    image_count = len(all_image_path)

    return dataset, image_count


def generate_datasets():
    
    # 生成数据
    train_dataset, train_count = get_dataset(dataset_root_dir=train_dir)
    valid_dataset, valid_count = get_dataset(dataset_root_dir=valid_dir)
    test_dataset, test_count = get_dataset(dataset_root_dir=test_dir)


    # 以BATCH_SIZE大小随机读取数据
    train_dataset = train_dataset.shuffle(buffer_size=train_count).batch(batch_size=BATCH_SIZE)
    valid_dataset = valid_dataset.batch(batch_size=BATCH_SIZE)
    test_dataset = test_dataset.batch(batch_size=BATCH_SIZE)

    return train_dataset, valid_dataset, test_dataset, train_count, valid_count, test_count

In [8]:
train_dataset, valid_dataset, test_dataset, train_count, valid_count, test_count = generate_datasets()

### 模型编写

<img src="./img/17.PNG" alt="FAO" width="600">
<img src="./img/18.PNG" alt="FAO" width="600">

### Block 核心块代码编写

In [9]:
class BottleNeck(tf.keras.layers.Layer):
    def __init__(self, filter_num, stride=1,with_other=True):
        super(BottleNeck, self).__init__()
        self.with_other = with_other
        
        # 2D卷积神经网络
        self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding='same')
        
        # 列归一化
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=stride,
                                            padding='same')
        
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv3 = tf.keras.layers.Conv2D(filters=filter_num * 4,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding='same')
        
        self.bn3 = tf.keras.layers.BatchNormalization()
        self.other = tf.keras.Sequential()
        self.other.add(tf.keras.layers.Conv2D(filters=filter_num * 4,
                                                   kernel_size=(1, 1),
                                                   strides=stride))
        self.other.add(tf.keras.layers.BatchNormalization())


    def call(self, inputs, training=None):
        
        covn_ = self.other(inputs)
        
        conv1 = self.conv1(inputs)
        
        # 训练时 需要batch，预测试时关闭，预测时候是全局的
        bn1 = self.bn1(conv1, training=training)
        relu1 = tf.nn.relu(bn1)
        conv2 = self.conv2(relu1)
        bn2 = self.bn2(conv2, training=training)
        relu2 = tf.nn.relu(bn2)
        conv3 = self.conv3(relu2)
        bn3 = self.bn3(conv3, training=training)
        
        if self.with_other == True:
            # 构建identity网络，加入了1*1 卷积合并特征
            output = tf.nn.relu(tf.keras.layers.add([covn_, bn3]))
        else:
            output = tf.nn.relu(tf.keras.layers.add([inputs, bn3]))

        return output


def build_res_block(filter_num, blocks, stride=1):
    res_block = tf.keras.Sequential()
    res_block.add(BottleNeck(filter_num, stride=stride, with_other=True))

    # 通过other 变量控制增加额外卷积，特征数量一致时无需增加
    for _ in range(1, blocks):
        res_block.add(BottleNeck(filter_num, stride=1,with_other=False))    
        
    return res_block

### ResNet50主模型编写 （混合模型编写）

In [10]:
class ResNet50(tf.keras.Model):
    def __init__(self, num_classes=NUM_CLASSES):
        super(ResNet50, self).__init__()

        # stage1
        self.pre1 = tf.keras.layers.Conv2D(filters=64,
                                           kernel_size=(7, 7),
                                           strides=2,
                                           padding='same')
        self.pre2 = tf.keras.layers.BatchNormalization()
        self.pre3 = tf.keras.layers.Activation(tf.keras.activations.relu)
        self.pre4 = tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                              strides=2)

        # stage2~stage5 特征数量和block数据发生了变化，都是以实验结果为导向
        # stage2
        self.layer1 = build_res_block(filter_num=64,
                                        blocks=3)
        
        # stage3
        self.layer2 = build_res_block(filter_num=128,
                                        blocks=4,
                                        stride=2)
        # stage4
        self.layer3 = build_res_block(filter_num=256,
                                        blocks=6,
                                        stride=2)
        
        # stage5
        self.layer4 = build_res_block(filter_num=512,
                                        blocks=3,
                                        stride=2)

        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        
        # 激活函数relu
        self.fc1 = tf.keras.layers.Dense(units=1000, activation=tf.keras.activations.relu)
        self.drop_out = tf.keras.layers.Dropout(rate=0.5)
        
        # 激活函数 softmax 主要用于多分类，进行1000分类
        self.fc2 = tf.keras.layers.Dense(units=num_classes, activation=tf.keras.activations.softmax)

    def call(self, inputs, training=None, mask=None):
        # 主模型流程
        
        # stage1
        pre1 = self.pre1(inputs)
        pre2 = self.pre2(pre1, training=training)
        pre3 = self.pre3(pre2)
        pre4 = self.pre4(pre3)
        
        # stage2
        l1 = self.layer1(pre4, training=training)
        
        # stage3
        l2 = self.layer2(l1, training=training)
        
        # stage4
        l3 = self.layer3(l2, training=training)
        
        # stage5
        l4 = self.layer4(l3, training=training)
        
        # 最后输出
        avgpool = self.avgpool(l4)
        fc1 = self.fc1(avgpool)
        drop = self.drop_out(fc1)
        out = self.fc2(drop)

        return out

In [11]:
model = ResNet50()
model.build(input_shape=(None, image_height, image_width, channels))
model.summary()

Model: "res_net50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  9472      
_________________________________________________________________
batch_normalization (BatchNo multiple                  256       
_________________________________________________________________
activation (Activation)      multiple                  0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
sequential (Sequential)      (None, 7, 7, 256)         353664    
_________________________________________________________________
sequential_4 (Sequential)    (None, 4, 4, 512)         2024448   
_________________________________________________________________
sequential_9 (Sequential)    (None, 2, 2, 1024)        12

### 定义loss 优化器 评价函数
- SparseCategoricalCrossentropy 对稀疏的多分类损失函数（类型特别多）
- SparseCategoricalAccuracy 对系数的多分类准确率
- Sparse 输出是整数，对应label的index 
- CategoricalCrossentropy 输出的one hot 对应的是[1,0,0]

In [12]:
# 设置loss 目标
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

# 设置优化器
optimizer = tf.keras.optimizers.Adam(lr=0.001)

# 设置train loss
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

# 设置valid loss
valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

### 分离式自定义求梯度， loss， 准确率，等

In [13]:
@tf.function
def train_step(images, labels):
    # 自定义梯度
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        
        # 使用自定义loss
        loss = loss_object(y_true=labels, y_pred=predictions)
        
    # 求出梯度结果
    gradients = tape.gradient(loss, model.trainable_variables)
    
    # 将梯度传到训练的所有参数变量中
    optimizer.apply_gradients(grads_and_vars=zip(gradients, model.trainable_variables))

    # 获取自定义的评价结果
    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def valid_step(images, labels):
    # 预测时候改变参数
    predictions = model(images, training=False)
    v_loss = loss_object(labels, predictions)

    # 获取自定义的评价结果
    valid_loss(v_loss)
    valid_accuracy(labels, predictions)

### 开始训练

In [14]:
for epoch in range(EPOCHS):
    
    # 重置所有梯度值
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()
    
    step = 0
    
    # 开始训练
    for images, labels in train_dataset:
        step += 1
        train_step(images, labels)
        print("Epoch: {}/{}, step: {}/{}, loss: {:.5f}, accuracy: {:.5f}".format(epoch + 1, EPOCHS,
                                                                                 step,
                                                                                 math.ceil(train_count / BATCH_SIZE),
                                                                                 train_loss.result(),
                                                                                 train_accuracy.result()))

Epoch: 1/10, step: 1/94, loss: 2.03080, accuracy: 0.40625
Epoch: 1/10, step: 2/94, loss: 3.93187, accuracy: 0.42188
Epoch: 1/10, step: 3/94, loss: 3.66513, accuracy: 0.43750
Epoch: 1/10, step: 4/94, loss: 5.35462, accuracy: 0.39844
Epoch: 1/10, step: 5/94, loss: 5.48107, accuracy: 0.38125
Epoch: 1/10, step: 6/94, loss: 5.35890, accuracy: 0.40104
Epoch: 1/10, step: 7/94, loss: 5.13128, accuracy: 0.39286
Epoch: 1/10, step: 8/94, loss: 5.10002, accuracy: 0.38672
Epoch: 1/10, step: 9/94, loss: 4.84424, accuracy: 0.40278
Epoch: 1/10, step: 10/94, loss: 5.25644, accuracy: 0.39687
Epoch: 1/10, step: 11/94, loss: 5.19930, accuracy: 0.39489
Epoch: 1/10, step: 12/94, loss: 4.93942, accuracy: 0.39062
Epoch: 1/10, step: 13/94, loss: 4.72257, accuracy: 0.38462
Epoch: 1/10, step: 14/94, loss: 4.52717, accuracy: 0.39286
Epoch: 1/10, step: 15/94, loss: 4.35087, accuracy: 0.39583
Epoch: 1/10, step: 16/94, loss: 4.15866, accuracy: 0.40430
Epoch: 1/10, step: 17/94, loss: 4.13516, accuracy: 0.40074
Epoch:

Epoch: 1/10, step: 93/94, loss: 2.01525, accuracy: 0.50336
Epoch: 1/10, step: 94/94, loss: 2.00381, accuracy: 0.50450
Epoch: 2/10, step: 1/94, loss: 1.00111, accuracy: 0.53125
Epoch: 2/10, step: 2/94, loss: 0.85897, accuracy: 0.64062
Epoch: 2/10, step: 3/94, loss: 0.83433, accuracy: 0.61458
Epoch: 2/10, step: 4/94, loss: 0.84749, accuracy: 0.60156
Epoch: 2/10, step: 5/94, loss: 0.83003, accuracy: 0.62500
Epoch: 2/10, step: 6/94, loss: 0.81225, accuracy: 0.63021
Epoch: 2/10, step: 7/94, loss: 0.79752, accuracy: 0.62500
Epoch: 2/10, step: 8/94, loss: 0.78537, accuracy: 0.63281
Epoch: 2/10, step: 9/94, loss: 0.80299, accuracy: 0.63194
Epoch: 2/10, step: 10/94, loss: 0.78532, accuracy: 0.63125
Epoch: 2/10, step: 11/94, loss: 0.80646, accuracy: 0.62216
Epoch: 2/10, step: 12/94, loss: 0.80133, accuracy: 0.61979
Epoch: 2/10, step: 13/94, loss: 0.80446, accuracy: 0.62019
Epoch: 2/10, step: 14/94, loss: 0.80452, accuracy: 0.62054
Epoch: 2/10, step: 15/94, loss: 0.80943, accuracy: 0.60833
Epoch:

Epoch: 3/10, step: 21/94, loss: 0.73004, accuracy: 0.63690
Epoch: 3/10, step: 22/94, loss: 0.72464, accuracy: 0.64062
Epoch: 3/10, step: 23/94, loss: 0.71898, accuracy: 0.64266
Epoch: 3/10, step: 24/94, loss: 0.71216, accuracy: 0.64193
Epoch: 3/10, step: 25/94, loss: 0.71359, accuracy: 0.64375
Epoch: 3/10, step: 26/94, loss: 0.70808, accuracy: 0.64543
Epoch: 3/10, step: 27/94, loss: 0.70323, accuracy: 0.64699
Epoch: 3/10, step: 28/94, loss: 0.69695, accuracy: 0.65067
Epoch: 3/10, step: 29/94, loss: 0.69021, accuracy: 0.65194
Epoch: 3/10, step: 30/94, loss: 0.70212, accuracy: 0.65208
Epoch: 3/10, step: 31/94, loss: 0.69847, accuracy: 0.65121
Epoch: 3/10, step: 32/94, loss: 0.69761, accuracy: 0.65234
Epoch: 3/10, step: 33/94, loss: 0.69676, accuracy: 0.65246
Epoch: 3/10, step: 34/94, loss: 0.69552, accuracy: 0.65074
Epoch: 3/10, step: 35/94, loss: 0.69774, accuracy: 0.65179
Epoch: 3/10, step: 36/94, loss: 0.69552, accuracy: 0.65278
Epoch: 3/10, step: 37/94, loss: 0.69012, accuracy: 0.657

Epoch: 4/10, step: 67/94, loss: 0.80455, accuracy: 0.63993
Epoch: 4/10, step: 68/94, loss: 0.80339, accuracy: 0.63971
Epoch: 4/10, step: 69/94, loss: 0.79990, accuracy: 0.64040
Epoch: 4/10, step: 70/94, loss: 0.79672, accuracy: 0.64241
Epoch: 4/10, step: 71/94, loss: 0.79941, accuracy: 0.64040
Epoch: 4/10, step: 72/94, loss: 0.79725, accuracy: 0.64062
Epoch: 4/10, step: 73/94, loss: 0.79657, accuracy: 0.63870
Epoch: 4/10, step: 74/94, loss: 0.79542, accuracy: 0.63936
Epoch: 4/10, step: 75/94, loss: 0.79700, accuracy: 0.63875
Epoch: 4/10, step: 76/94, loss: 0.79564, accuracy: 0.63939
Epoch: 4/10, step: 77/94, loss: 0.79528, accuracy: 0.63839
Epoch: 4/10, step: 78/94, loss: 0.79456, accuracy: 0.63742
Epoch: 4/10, step: 79/94, loss: 0.79317, accuracy: 0.63766
Epoch: 4/10, step: 80/94, loss: 0.79200, accuracy: 0.63789
Epoch: 4/10, step: 81/94, loss: 0.79150, accuracy: 0.63619
Epoch: 4/10, step: 82/94, loss: 0.79018, accuracy: 0.63605
Epoch: 4/10, step: 83/94, loss: 0.78852, accuracy: 0.637

Epoch: 6/10, step: 19/94, loss: 0.60090, accuracy: 0.73026
Epoch: 6/10, step: 20/94, loss: 0.59858, accuracy: 0.73125
Epoch: 6/10, step: 21/94, loss: 0.59527, accuracy: 0.72768
Epoch: 6/10, step: 22/94, loss: 0.60898, accuracy: 0.71307
Epoch: 6/10, step: 23/94, loss: 0.60764, accuracy: 0.71467
Epoch: 6/10, step: 24/94, loss: 0.60733, accuracy: 0.71615
Epoch: 6/10, step: 25/94, loss: 0.61138, accuracy: 0.71750
Epoch: 6/10, step: 26/94, loss: 0.60970, accuracy: 0.71995
Epoch: 6/10, step: 27/94, loss: 0.60772, accuracy: 0.71759
Epoch: 6/10, step: 28/94, loss: 0.60428, accuracy: 0.72098
Epoch: 6/10, step: 29/94, loss: 0.60852, accuracy: 0.71983
Epoch: 6/10, step: 30/94, loss: 0.60790, accuracy: 0.72083
Epoch: 6/10, step: 31/94, loss: 0.60442, accuracy: 0.72278
Epoch: 6/10, step: 32/94, loss: 0.60382, accuracy: 0.72266
Epoch: 6/10, step: 33/94, loss: 0.60340, accuracy: 0.72348
Epoch: 6/10, step: 34/94, loss: 0.59575, accuracy: 0.72886
Epoch: 6/10, step: 35/94, loss: 0.60257, accuracy: 0.727

Epoch: 7/10, step: 65/94, loss: 0.46667, accuracy: 0.80481
Epoch: 7/10, step: 66/94, loss: 0.47068, accuracy: 0.80398
Epoch: 7/10, step: 67/94, loss: 0.47106, accuracy: 0.80271
Epoch: 7/10, step: 68/94, loss: 0.47027, accuracy: 0.80331
Epoch: 7/10, step: 69/94, loss: 0.46836, accuracy: 0.80480
Epoch: 7/10, step: 70/94, loss: 0.47217, accuracy: 0.80179
Epoch: 7/10, step: 71/94, loss: 0.47356, accuracy: 0.80106
Epoch: 7/10, step: 72/94, loss: 0.47247, accuracy: 0.80078
Epoch: 7/10, step: 73/94, loss: 0.47042, accuracy: 0.80180
Epoch: 7/10, step: 74/94, loss: 0.47318, accuracy: 0.79983
Epoch: 7/10, step: 75/94, loss: 0.47236, accuracy: 0.80083
Epoch: 7/10, step: 76/94, loss: 0.47315, accuracy: 0.80140
Epoch: 7/10, step: 77/94, loss: 0.47118, accuracy: 0.80276
Epoch: 7/10, step: 78/94, loss: 0.47343, accuracy: 0.80248
Epoch: 7/10, step: 79/94, loss: 0.47451, accuracy: 0.80182
Epoch: 7/10, step: 80/94, loss: 0.47398, accuracy: 0.80156
Epoch: 7/10, step: 81/94, loss: 0.47721, accuracy: 0.800

Epoch: 9/10, step: 17/94, loss: 0.27007, accuracy: 0.88971
Epoch: 9/10, step: 18/94, loss: 0.26386, accuracy: 0.89236
Epoch: 9/10, step: 19/94, loss: 0.25997, accuracy: 0.89638
Epoch: 9/10, step: 20/94, loss: 0.26434, accuracy: 0.89062
Epoch: 9/10, step: 21/94, loss: 0.26349, accuracy: 0.88988
Epoch: 9/10, step: 22/94, loss: 0.25668, accuracy: 0.89347
Epoch: 9/10, step: 23/94, loss: 0.26223, accuracy: 0.89402
Epoch: 9/10, step: 24/94, loss: 0.26024, accuracy: 0.89323
Epoch: 9/10, step: 25/94, loss: 0.25216, accuracy: 0.89625
Epoch: 9/10, step: 26/94, loss: 0.25850, accuracy: 0.89423
Epoch: 9/10, step: 27/94, loss: 0.26197, accuracy: 0.89236
Epoch: 9/10, step: 28/94, loss: 0.27482, accuracy: 0.88728
Epoch: 9/10, step: 29/94, loss: 0.26915, accuracy: 0.88901
Epoch: 9/10, step: 30/94, loss: 0.27214, accuracy: 0.88854
Epoch: 9/10, step: 31/94, loss: 0.27929, accuracy: 0.88911
Epoch: 9/10, step: 32/94, loss: 0.30465, accuracy: 0.88184
Epoch: 9/10, step: 33/94, loss: 0.30650, accuracy: 0.882

Epoch: 10/10, step: 61/94, loss: 0.30777, accuracy: 0.88832
Epoch: 10/10, step: 62/94, loss: 0.30648, accuracy: 0.88911
Epoch: 10/10, step: 63/94, loss: 0.31120, accuracy: 0.88889
Epoch: 10/10, step: 64/94, loss: 0.31259, accuracy: 0.88867
Epoch: 10/10, step: 65/94, loss: 0.31307, accuracy: 0.88798
Epoch: 10/10, step: 66/94, loss: 0.31479, accuracy: 0.88778
Epoch: 10/10, step: 67/94, loss: 0.31484, accuracy: 0.88759
Epoch: 10/10, step: 68/94, loss: 0.31461, accuracy: 0.88787
Epoch: 10/10, step: 69/94, loss: 0.31499, accuracy: 0.88587
Epoch: 10/10, step: 70/94, loss: 0.31531, accuracy: 0.88482
Epoch: 10/10, step: 71/94, loss: 0.31475, accuracy: 0.88512
Epoch: 10/10, step: 72/94, loss: 0.31597, accuracy: 0.88411
Epoch: 10/10, step: 73/94, loss: 0.31767, accuracy: 0.88271
Epoch: 10/10, step: 74/94, loss: 0.31581, accuracy: 0.88345
Epoch: 10/10, step: 75/94, loss: 0.31573, accuracy: 0.88333
Epoch: 10/10, step: 76/94, loss: 0.31689, accuracy: 0.88240
Epoch: 10/10, step: 77/94, loss: 0.31894

### 储存模型

In [15]:
model.save_weights(filepath=save_model_dir, save_format='tf')

### 重新加载模型

In [16]:
model = ResNet50()
model.build(input_shape=(None, image_height, image_width, channels))
model.load_weights(filepath=save_model_dir)

### 在测试数据集测试结果

In [17]:
# 在测试数据重复之前的操作
loss_object = tf.keras.metrics.SparseCategoricalCrossentropy()
test_loss = tf.keras.metrics.Mean()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

for test_images, test_labels in test_dataset:
    test_step(test_images, test_labels)
#     print("loss: {:.5f}, test accuracy: {:.5f}".format(test_loss.result(),
#                                                        test_accuracy.result()))

print("The accuracy on test set is: {:.3f}%".format(test_accuracy.result()*100))

The accuracy on test set is: 90.869%
